# 프로젝트: SentencePiece 사용하기

## Step 1. SentencePiece 설치하기

In [1]:
! pip install sentencepiece

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


## Step 2. SentencePiece 모델 학습

In [2]:
import sentencepiece as spm
import os
temp_file = os.getenv('HOME')+'/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp'

vocab_size = 8000

with open(temp_file, 'w') as f:
    for row in filtered_corpus:   # 이전 스텝에서 정제했던 corpus를 활용합니다.
        f.write(str(row) + '\n')

spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix=korean_spm --vocab_size={}'.format(temp_file, vocab_size)    
)
#위 Train에서  --model_type = 'unigram'이 디폴트 적용되어 있습니다. --model_type = 'bpe' 로 옵션을 주어 변경할 수 있습니다.

!ls -l korean_spm*

NameError: name 'filtered_corpus' is not defined

In [ ]:
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

# SentencePiece를 활용한 sentence -> encoding
tokensIDs = s.EncodeAsIds('아버지가방에들어가신다.')
print(tokensIDs)

# SentencePiece를 활용한 sentence -> encoded pieces
print(s.SampleEncodeAsPieces('아버지가방에들어가신다.',1, 0.0))

# SentencePiece를 활용한 encoding -> sentence 복원
print(s.DecodeIds(tokensIDs))

## Step 3. Tokenizer 함수 작성

In [ ]:
def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({idx:word})
        index_word.update({word:idx})

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word

In [ ]:
#sp_tokenize(s, corpus) 사용예제

my_corpus = ['나는 밥을 먹었습니다.', '그러나 여전히 ㅠㅠ 배가 고픕니다...']
tensor, word_index, index_word = sp_tokenize(s, my_corpus)
print(tensor)

### Step 4. 네이버 영화리뷰 감정 분석 문제에 SentencePiece 적용해 보기

In [ ]:
#폴더생성
!mkdir ~/aiffel/sp_tokenizer/sentiment_classification

In [ ]:
#리뷰데이터 받아오기
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt  
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!mv ratings_*.txt ~/aiffel/sp_tokenizer/sentiment_classification 

[추가 학습]
- ! wget + DOWNLOAD-URL  : 단일 파일받기
- ! mv 파일명 + 이동경로 : 하나 이상의 파일이나 디렉터리를 한 장소에서 한 다른 장소로 이동하도록 만드는 유닉스 명령어

In [ ]:
# 1. 데이터 준비와 확인

import pandas as pd
import urllib.request
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import re

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

# 데이터를 읽기
train_data = pd.read_table('~/aiffel/sp_tokenizer/sentiment_classification/ratings_train.txt') #경로 변경 완료
test_data = pd.read_table('~/aiffel/sp_tokenizer/sentiment_classification/ratings_test.txt')   #경로 변경 완료

display(train_data.head(6))
display(test_data.head(6))
print("훈련 데이터 개수 : {}, 테스트 데이터 개수: {}".format(len(train_data), len(test_data)))

#### 1) 학습  및 테스트 데이터 중복 제거

In [ ]:
train_data.drop_duplicates(subset=['document'], inplace=True)
test_data.drop_duplicates(subset=['document'], inplace=True)

print('중복제거 후 학습데이터 문장의 개수 : ',len(train_data['document']))
print('중복제거 후 테스트데이터 문장의 개수 : ',len(test_data['document']))

#### 2) 학습 및 테스트 데이터 Nan 결측치 제거

In [ ]:
train_data = train_data.dropna(how = 'any')
test_data =test_data.dropna(how='any')

print('결측치제거 후 학습데이터 문장의 개수 : ',len(train_data['document']))
print('결측치제거 후 테스트데이터 문장의 개수 : ',len(test_data['document']))

[중복제거]

- ex 노드는 중복값  drop_duplicates 메서드 이용하여 제거 , train_data.drop_duplicates(subset=['document'], inplace=True)

- nlp 노드는 중복제거시 cleaned_corpus = list(set(raw))  # set를 사용해서 중복을 제거합니다.
    - set은 집합을 정의하는 자료형인데, 중복을 허용하지 않아 변환 과정에서 자동으로 중복된 요소를 제거
    - 대신 list의 순서가 뒤죽박죽될 수 있으니, 만약 번역 데이터처럼 쌍을 이뤄야 하는 경우라면 주의해서 사용

좀 햇갈리는 부분 이다.

#### 3) 문장 corpus 분석

In [ ]:
#  문장의 최단 길이, 최장 길이, 평균 길이를 구한 후 문장 길이 분포를 막대그래프로 표현해 주는 소스

min_len = 999
max_len = 0
sum_len = 0

for sen in train_data['document']:  # train_data['document'] 앞서 다운로드받은 데이터가 담긴 변수
    length = len(sen)
    if min_len > length: min_len = length
    if max_len < length: max_len = length
    sum_len += length

print("Data Size:", len(train_data['document']))
print("문장의 최단 길이:", min_len)
print("문장의 최장 길이:", max_len)
print("문장의 평균 길이:", sum_len // len(train_data['document']))

sentence_length = np.zeros((max_len), dtype=np.int) # np.zeros 주어진 형태와 타입을 갖는 0으로 채워진 어레이를 반환

for sen in train_data['document']:   
    sentence_length[len(sen)-1] += 1

plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("train_data Sentence Length Distribution")
plt.show()

In [ ]:
# 정제된 데이터를 공백 기반으로 토큰화하여 저장

cleaned_corpus = []
for sen in train_data['document']:
    cleaned_corpus.append(sen)

In [ ]:
max_len = 60
min_len = 5 # 너무 짧은 데이터는 노이즈로 작용할수 있으므로 길이 5미만 제거

# 길이 조건에 맞는 문장만 선택합니다.
filtered_corpus = [s for s in cleaned_corpus if (len(s) < max_len) & (len(s) >= min_len)]

# 분포도를 다시 그려봅니다.
sentence_length = np.zeros((max_len), dtype=np.int)

for sen in filtered_corpus:
    sentence_length[len(sen)-1] += 1

plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("Sentence Length Distribution")
plt.show()

### 1) 네이버 영화리뷰 감정 분석 코퍼스에 SentencePiece를 적용시킨 모델 학습하기

In [ ]:
import sentencepiece as spm
import os
temp_file = os.getenv('HOME')+'/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp'

vocab_size = 8000

with open(temp_file, 'w') as f:
    for row in filtered_corpus:   # 이전 스텝에서 정제했던 corpus를 활용합니다.
        f.write(str(row) + '\n')

spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix=korean_spm --vocab_size={}'.format(temp_file, vocab_size)    
)
#위 Train에서  --model_type = 'unigram'이 디폴트 적용되어 있습니다. --model_type = 'bpe' 로 옵션을 주어 변경할 수 있습니다.

!ls -l korean_spm*

### 2) 학습된 모델로 sp_tokenize() 메소드 구현하기

In [ ]:
s = spm.SentencePieceProcessor()
print(s.Load('korean_spm.model'))

def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({idx:word})
        index_word.update({word:idx})

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word

### 3) 구현된 토크나이저를 적용하여 네이버 영화리뷰 감정 분석 모델을 재학습하기

In [ ]:
# 모델 설계
vocab_size = 10000    # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 41  # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다.

In [ ]:
# 1) LSTM 모델
model = keras.Sequential(name='LSTM')
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(keras.layers.LSTM(8))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경 가능)
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

In [ ]:
# 모델 학습

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_input, train_target, epochs=10, validation_data=(val_input, val_target))

In [ ]:
# 모델 테스트셋으로 평가  
results = model.evaluate(X_test,  y_test, verbose=2)

print(results)

#### KoNLPy 형태소 분석기를 사용한 모델과 성능 비교하기

#### (보너스) SentencePiece 모델의 model_type, vocab_size 등을 변경해 가면서 성능 개선 여부 확인하기

In [ ]:
코퍼스 분석, 전처리, SentencePiece 적용, 토크나이저 구현 및 동작이 빠짐없이 진행되었는가

In [ ]:
SentencePiece 토크나이저가 적용된 Text Classifier 모델이 정상적으로 수렴하여 80% 이상의 test accuracy가 확인되었다.

In [ ]:
SentencePiece 토크나이저를 활용했을 때의 성능을 다른 토크나이저 혹은 SentencePiece의 다른 옵션의 경우와 비교하여 분석을 체계적으로 진행하였다